In [16]:
# ─────────────────────────────────────────────────────────────
# 1)  Imports, paths, and constants
# ─────────────────────────────────────────────────────────────
import os, re, pandas as pd
from typing import List, Tuple, Dict

# <<< TWO PATHS TO MATCH FOLDERS >>>
input_dir  = r"/Users/unmonadas/Desktop/ALL PROJECT FILES/WIDE CSV/OneDrive_1_5-5-2025"
output_dir = r"/Users/unmonadas/Desktop/CSV WIDE FINAL1"

FINAL_COLUMNS: List[str] = [
    "hospital_name", "street_address", "city", "state", "zip_code",
    "description", "billing_code", "billing_code_type",
    "standard_charge", "discounted_cash_charge",
    "payer_name", "plan_name", "negotiated_dollar", "negotiated_percentage",
    "estimated_amount", "min_charge", "max_charge",
]
CPT_TYPE        = "CPT"
CODE_IDX_FOR_CPT = 3  # CPT code lives in code|3


In [17]:
# ─────────────────────────────────────────────────────────────
# 2)  Helper: robust address parser
# ─────────────────────────────────────────────────────────────
def parse_address(addr: str) -> Tuple[str, str, str, str]:
    """Return (street, city, state, zip) or blanks on failure."""
    if not isinstance(addr, str) or not addr.strip():
        return "", "", "", ""
    addr = " ".join(addr.replace("\n", " ").split())               # collapse whitespace
    addr = re.sub(r"\s+,", ",", addr)                              # strip space before commas
    m = re.search(r"([A-Z]{2})\s*(\d{5})(?:-\d{4})?$", addr)       # FL 32935 (optionally -1234)
    if not m:
        return "", "", "", ""
    state, zip_code = m.group(1), m.group(2)
    left = addr[: m.start()].rstrip(", ")
    if "," in left:
        street, city = map(str.strip, left.rsplit(",", 1))
    else:
        parts = left.split()
        city   = parts[-1] if len(parts) >= 2 else ""
        street = " ".join(parts[:-1]) if len(parts) >= 2 else left
    return street, city, state, zip_code


In [18]:
# ─────────────────────────────────────────────────────────────
# 3)  Core cleaner for ONE wide CSV
# ─────────────────────────────────────────────────────────────
def clean_wide_csv(path: str) -> pd.DataFrame:
    raw = pd.read_csv(path, low_memory=False)
    
    hosp_name = str(raw.loc[0, "hospital_name"]).strip()
    street, city, state, zip_code = parse_address(str(raw.loc[0, "hospital_address"]))
    
    header = raw.iloc[1].fillna(method="ffill")
    data   = raw.iloc[2:].copy()
    data.columns = header
    data.reset_index(drop=True, inplace=True)
    
    cpt = data[data[f"code|{CODE_IDX_FOR_CPT}|type"].str.upper() == CPT_TYPE].copy()
    if cpt.empty:
        return pd.DataFrame(columns=FINAL_COLUMNS)
    
    cpt = cpt.rename(columns={
        "description": "description",
        f"code|{CODE_IDX_FOR_CPT}": "billing_code",
        f"code|{CODE_IDX_FOR_CPT}|type": "billing_code_type",
        "standard_charge|gross": "standard_charge",
        "standard_charge|discounted_cash": "discounted_cash_charge",
        "standard_charge|min": "min_charge",
        "standard_charge|max": "max_charge",
    })
    for col in ["standard_charge", "discounted_cash_charge", "min_charge", "max_charge"]:
        cpt[col] = pd.to_numeric(cpt[col], errors="coerce")
    
    # collect negotiated columns
    pat = re.compile(r"standard_charge\|([^|]+)\|([^|]+)\|negotiated_(dollar|percentage)")
    payer_cols: Dict[Tuple[str, str], Dict[str, str]] = {}
    for col in cpt.columns:
        m = pat.match(col)
        if m:
            payer, plan, kind = m.groups()
            payer_cols.setdefault((payer.strip(), plan.strip()), {})[kind] = col
    
    rows = []
    for _, r in cpt.iterrows():
        for (payer, plan), mcols in payer_cols.items():
            d_val = r.get(mcols.get("dollar"))
            p_val = r.get(mcols.get("percentage"))
            if pd.isna(d_val) and pd.isna(p_val):
                continue
            rows.append({
                "hospital_name": hosp_name,
                "street_address": street,
                "city": city,
                "state": state,
                "zip_code": zip_code,
                "description": r["description"],
                "billing_code": r["billing_code"],
                "billing_code_type": CPT_TYPE,
                "standard_charge": r["standard_charge"],
                "discounted_cash_charge": r["discounted_cash_charge"],
                "payer_name": payer,
                "plan_name": plan,
                "negotiated_dollar": pd.to_numeric(d_val, errors="coerce"),
                "negotiated_percentage": pd.to_numeric(p_val, errors="coerce"),
                "estimated_amount": None,
                "min_charge": r["min_charge"],
                "max_charge": r["max_charge"],
            })
    return pd.DataFrame(rows, columns=FINAL_COLUMNS)


In [19]:
# ─────────────────────────────────────────────────────────────
# 4)  Batch‑clean every CSV in input_dir
# ─────────────────────────────────────────────────────────────
os.makedirs(output_dir, exist_ok=True)

for fname in [f for f in os.listdir(input_dir) if f.lower().endswith(".csv")]:
    print(f"→ Processing {fname} …", end=" ")
    df_clean = clean_wide_csv(os.path.join(input_dir, fname))
    out_name = os.path.splitext(fname)[0] + "_cleaned.csv"
    df_clean.to_csv(os.path.join(output_dir, out_name), index=False)
    print(f"done (rows: {len(df_clean)})")


→ Processing 994762475_SebastianRiverMedicalCenter_standardcharges.csv … done (rows: 324019)
→ Processing 994762293_RockledgeRegionalMedicalCenter_standardcharges.csv … done (rows: 319748)
→ Processing 994762293_MelbourneRegionalMedicalCenter_standardcharges.csv … done (rows: 287464)
